# Regularización en Machine Learning

<a target="_blank" href="https://colab.research.google.com/github/griverat/Meteo-AI/blob/main/notebooks/2.regularization.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Si usa Google Colab, asegúrese de tener habilitada la GPU para este notebook.**

![gpu_colab](../images/colab_gpu.png)

## Descripción

En este notebook vamos a ver cómo la regularización puede ayudar a mejorar el rendimiento de un modelo de Machine Learning. Para ello, vamos a utilizar un dataset de ejemplo y vamos a entrenar un modelo sin regularización y otro con regularización, explorando cómo se comportan ambos modelos.

## Objetivos

- Entender qué es la regularización y cómo se aplica en Machine Learning.
- Comparar distintos mecanismos de regularización.
- Entrenar un modelo con y sin regularización.

---

## ¿Qué es la regularización?

La regularización es una técnica utilizada en Machine Learning para evitar el sobreajuste de los modelos. El sobreajuste ocurre cuando un modelo se ajusta demasiado bien a los datos de entrenamiento, lo que puede llevar a que el modelo tenga un rendimiento deficiente en datos nuevos o no vistos.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/19/Overfitting.svg/1280px-Overfitting.svg.png" width="400"></img>

La línea verde representa un modelo sobreajustado y la línea negra representa un modelo regularizado. Si bien la línea verde sigue mejor los datos de entrenamiento, depende demasiado de esos datos y es probable que tenga una tasa de error más alta en datos nuevos no vistos, ilustrados por puntos delineados en negro, en comparación con la línea negra.


Existen diferentes métodos de regularización que se pueden aplicar a los modelos de aprendizaje automático. Dos de los métodos más comunes son la regularización L1 y la regularización L2.

La regularización L1, también conocida como regularización de Lasso, agrega una penalización a la función de costo del modelo basada en la suma de los valores absolutos de los coeficientes del modelo. Esto tiene el efecto de forzar algunos coeficientes a cero, lo que puede conducir a la selección automática de características y simplificación del modelo.

La regularización L2, también conocida como regularización de Ridge, agrega una penalización a la función de costo del modelo basada en la suma de los cuadrados de los coeficientes del modelo. Esto tiene el efecto de reducir los valores de los coeficientes, lo que puede ayudar a evitar el sobreajuste y mejorar la generalización del modelo.

Además de la regularización L1 y L2, también existen otros métodos de regularización, como la elastic net, que combina las penalizaciones de L1 y L2, y la regularización de dropout, que apaga aleatoriamente algunas neuronas durante el entrenamiento para evitar la dependencia excesiva de ciertas características.

La regularización es especialmente útil cuando se trabaja con conjuntos de datos pequeños o cuando hay una alta dimensionalidad de características. Al agregar una penalización a la función de costo, la regularización ayuda a controlar la complejidad del modelo y a evitar el sobreajuste, lo que puede resultar en modelos más estables y con mejor rendimiento en datos nuevos.

En resumen, la regularización es una técnica importante en el aprendizaje automático que ayuda a evitar el sobreajuste y mejorar la generalización del modelo. Al aplicar una penalización a la función de costo, la regularización controla la complejidad del modelo y promueve la selección de características relevantes, lo que puede conducir a modelos más estables y con mejor rendimiento en datos nuevos.

## Regularización en Tensorflow

La regularización implementada en tensorflow, mediante la interfaz de Keras, es aplicada independientemente por capas. Esto indica que dependiendo de como nos gustase configurar el modelo, podemos escoger distintas formas de regularización con distintas intensidades para cada capa de nuestro modelo.

En Keras, existen tres tipos de regularización que se pueden aplicar a las capas de un modelo: regularización del kernel, regularización del bias y regularización de la actividad.

1. Regularización del kernel:
La regularización del kernel se aplica a los pesos de las conexiones entre las neuronas de una capa. Ayuda a controlar la complejidad del modelo penalizando los valores grandes de los pesos. Esto se logra mediante la adición de una penalización a la función de pérdida del modelo. La regularización del kernel se puede utilizar para evitar el sobreajuste y mejorar la generalización del modelo.

2. Regularización del bias:
La regularización del bias se aplica a los términos de sesgo de las neuronas de una capa. Al igual que la regularización del kernel, la regularización del bias ayuda a controlar la complejidad del modelo y evitar el sobreajuste. Se puede utilizar para reducir la dependencia excesiva de ciertas características y mejorar la capacidad de generalización del modelo.

3. Regularización de la actividad:
La regularización de la actividad se aplica a la salida de una capa. Ayuda a controlar la magnitud de las activaciones de las neuronas, evitando así valores extremadamente grandes o pequeños. Esto puede ser útil para evitar la saturación de las funciones de activación y mejorar la estabilidad del modelo.

Para aplicar la regularización en Keras, se pueden utilizar diferentes técnicas, como la regularización L1, la regularización L2 o la regularización de dropout. Estas técnicas se pueden especificar al definir una capa en Keras, utilizando los parámetros correspondientes, como `kernel_regularizer`, `bias_regularizer` y `activity_regularizer`.

```python
from tensorflow.keras import layers, regularizers

model = keras.Sequential([
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01),
                 bias_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l2(0.01)),
    layers.Dense(10, activation='softmax')
])
```

En este ejemplo, se utiliza la regularización L2 con un factor de penalización de 0.01 para la regularización del kernel, el bias y la actividad de la capa densa. Esto ayudará a controlar la complejidad del modelo y evitar el sobreajuste.

### Implementación

Para demostrar el efecto de la regularización en un modelo de Machine Learning, vamos a utilizar un dataset sintético creado usando `sklearn` para entrenar un modelo clasificación binaria. Al usar un set de datos sintético, podemos controlar el nivel de ruido y la complejidad del problema, lo que nos permitirá ver cómo la regularización actúa realmente

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.datasets import make_circles

In [ ]:
x, y = make_circles(n_samples=100, noise=0.1, random_state=1)

df = pd.DataFrame(dict(x=x[:, 0], y=x[:, 1], label=y))
colors = {0: "red", 1: "blue"}

fig, ax = plt.subplots()
grouped = df.groupby("label")
for key, group in grouped:
    group.plot(ax=ax, kind="scatter", x="x", y="y", label=key, color=colors[key])

Como siempre, antes de comenzar a definir los modelos a usar, debemos separar nuestros datos en al menos dos conjuntos: uno de entrenamiento y otro de prueba. Para este ejemplo vamos a exagerar y usar 30 ejemplos para entrenar y 70 para probar.



In [ ]:
x_train, x_test = x[:30], x[30:]
y_train, y_test = y[:30], y[30:]

## Sobreajuste de perceptrón multicapa

Vamos a desarrollar un perceptrón multicapa para resolver este problema de clasificación binaria. El modelo tendrá una sola capa con más neuronas de las necesarias para resolver el problema, lo que lo hará propenso a sobreajustarse a los datos de entrenamiento. Adicionalmente, entrenaremos el modelo durante un número de épocas mayor al necesario para que el sobreajuste sea más evidente.


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(2,)))
model.add(tf.keras.layers.Dense(500, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

El modelo definido va a set entrenado por 4000 epocas con un batch size de 32

In [ ]:
history = model.fit(
    x_train, y_train, validation_data=(x_test, y_test), epochs=4000, verbose=0
)

Podemos revisar las métricas de entrenamiento, en donde se nota claramente como el modelo se ajusta a los datos de entrenamiento, pero no logra generalizar bien a los datos de prueba.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
ax[0].plot(history.history["loss"], label="train")
ax[0].plot(history.history["val_loss"], label="test")
ax[0].legend()
ax[0].set_title("loss")

ax[1].plot(history.history["accuracy"], label="train")
ax[1].plot(history.history["val_accuracy"], label="test")
ax[1].legend()
ax[1].set_title("accuracy")

Podemos visualizar la frontera de decisión del modelo, la cual se ajusta muy bien a los datos de entrenamiento, pero no logra generalizar bien a los datos de prueba.

In [ ]:
grouded_train = pd.DataFrame(dict(x=x_train[:, 0], y=x_train[:, 1], label=y_train))
grouped_test = pd.DataFrame(dict(x=x_test[:, 0], y=x_test[:, 1], label=y_test))
grouped_train = grouded_train.groupby("label")
grouped_test = grouped_test.groupby("label")

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
for key, group in grouped_train:
    group.plot(ax=axs[0], kind="scatter", x="x", y="y", label=key, color=colors[key])
for key, group in grouped_test:
    group.plot(ax=axs[1], kind="scatter", x="x", y="y", label=key, color=colors[key])

for dset, ax in zip([x_train, x_test], axs):
    x_min, x_max = dset[:, 0].min() - 1, dset[:, 0].max() + 1
    y_min, y_max = dset[:, 1].min() - 1, dset[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])[:, 0]
    Z = Z.reshape(xx.shape)
    ax.contour(xx, yy, Z, levels=[0.5], linestyles="dashed")

axs[0].set_title("train")
axs[1].set_title("test")

In [ ]:
_, train_acc = model.evaluate(x_train, y_train, verbose=0)
_, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Train: {train_acc:.1%}, Test: {test_acc:.1%}")

## Regularización en el modelo sobreajustado

Ahora vamos a ver como los distinto algoritmos de regularización pueden ayudar a mejorar el rendimiento del modelo. Para ello, vamos a entrenar el mismo modelo, pero con regularización L1 y L2 en las capas densas.

Para comenzar, debemos usar una activación lineal en la capa densa de la red, ya que queremos aplicar la regularización sobre los datos en crudo, antes de pasar por la función de activación.


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(2,)))
model.add(
    tf.keras.layers.Dense(
        500, activation="linear", activity_regularizer=tf.keras.regularizers.l1(0.0001)
    )
)
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Ahora realizamos el entrenamiento para evaluar el cambio en el rendimiento del modelo.

In [ ]:
history = model.fit(
    x_train, y_train, validation_data=(x_test, y_test), epochs=4000, verbose=0
)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
ax[0].plot(history.history["loss"], label="train")
ax[0].plot(history.history["val_loss"], label="test")
ax[0].legend()
ax[0].set_title("loss")

ax[1].plot(history.history["accuracy"], label="train")
ax[1].plot(history.history["val_accuracy"], label="test")
ax[1].legend()
ax[1].set_title("accuracy")

In [ ]:
_, train_acc = model.evaluate(x_train, y_train, verbose=0)
_, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Train: {train_acc:.1%}, Test: {test_acc:.1%}")

In [ ]:
grouded_train = pd.DataFrame(dict(x=x_train[:, 0], y=x_train[:, 1], label=y_train))
grouped_test = pd.DataFrame(dict(x=x_test[:, 0], y=x_test[:, 1], label=y_test))
grouped_train = grouded_train.groupby("label")
grouped_test = grouped_test.groupby("label")

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
for key, group in grouped_train:
    group.plot(ax=axs[0], kind="scatter", x="x", y="y", label=key, color=colors[key])
for key, group in grouped_test:
    group.plot(ax=axs[1], kind="scatter", x="x", y="y", label=key, color=colors[key])

for dset, ax in zip([x_train, x_test], axs):
    x_min, x_max = dset[:, 0].min() - 1, dset[:, 0].max() + 1
    y_min, y_max = dset[:, 1].min() - 1, dset[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])[:, 0]
    Z = Z.reshape(xx.shape)
    ax.contour(xx, yy, Z, levels=[0.5], linestyles="dashed")

De manera similar, podemos hacer uso de la regularización L2 en la capa densa para ver cómo afecta al rendimiento del modelo.

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(2,)))
model.add(
    tf.keras.layers.Dense(
        500, activation="linear", activity_regularizer=tf.keras.regularizers.l2(0.0001)
    )
)
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Ahora realizamos el entrenamiento para evaluar el cambio en el rendimiento del modelo.

In [ ]:
history = model.fit(
    x_train, y_train, validation_data=(x_test, y_test), epochs=4000, verbose=0
)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
ax[0].plot(history.history["loss"], label="train")
ax[0].plot(history.history["val_loss"], label="test")
ax[0].legend()
ax[0].set_title("loss")

ax[1].plot(history.history["accuracy"], label="train")
ax[1].plot(history.history["val_accuracy"], label="test")
ax[1].legend()
ax[1].set_title("accuracy")

In [ ]:
_, train_acc = model.evaluate(x_train, y_train, verbose=0)
_, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Train: {train_acc:.1%}, Test: {test_acc:.1%}")

In [ ]:
grouded_train = pd.DataFrame(dict(x=x_train[:, 0], y=x_train[:, 1], label=y_train))
grouped_test = pd.DataFrame(dict(x=x_test[:, 0], y=x_test[:, 1], label=y_test))
grouped_train = grouded_train.groupby("label")
grouped_test = grouped_test.groupby("label")

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
for key, group in grouped_train:
    group.plot(ax=axs[0], kind="scatter", x="x", y="y", label=key, color=colors[key])
for key, group in grouped_test:
    group.plot(ax=axs[1], kind="scatter", x="x", y="y", label=key, color=colors[key])

for dset, ax in zip([x_train, x_test], axs):
    x_min, x_max = dset[:, 0].min() - 1, dset[:, 0].max() + 1
    y_min, y_max = dset[:, 1].min() - 1, dset[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])[:, 0]
    Z = Z.reshape(xx.shape)
    ax.contour(xx, yy, Z, levels=[0.5], linestyles="dashed")

Podemos verificar que pese a usar regularizacion L2 para este problema, el modelo sigue sobreajustando los datos de entrenamiento, indicando que es mejor cortar las conexiones con la regularización L1 que reducir su peso con la regularización L2.

### Ejercicio

Hemos realizado pruebas con el regularizador L1 y L2, ahora te toca a ti implementar la regularización ElasticNet (combinación de L1 y L2) para ver cómo afecta al rendimiento del modelo.

Keras proprociona la función `regularizers.L1L2`, la cual toma como parámetros `l1` y `l2`, los cuales son los factores de penalización para la regularización L1 y L2, respectivamente.

```python
tf.keras.regularizers.L1L2(l1=0.0, l2=0.0)
```

In [ ]:
# code here

## Dropout

Vamos a retomar uno de los ejemplos iniciales que realizamos en la introducción a Keras, en donde entrenamos un modelo de perceptrón multicapa para el pronóstico de la temperatura en la siguiente hora. En este caso, vamos a aplicar la técnica de dropout para evitar el sobreajuste del modelo.

In [ ]:
station_data = pd.read_csv("data/campo_de_marte.csv", skiprows=10)

# renombramos las columnas a algo más amigable
station_data.columns = [
    "date",
    "hour",
    "temp",
    "precip",
    "humidity",
    "wind_dir",
    "wind_speed",
]

# combinamos las columnas de fecha y hora en una sola
station_data["date"] = pd.to_datetime(station_data["date"] + " " + station_data["hour"])
station_data = station_data.drop(columns=["hour"])

# convertimos las columnas de temperatura, precipitación, humedad y velocidad del viento a números
station_data["temp"] = pd.to_numeric(
    station_data["temp"], errors="coerce", downcast="float"
)
station_data["precip"] = pd.to_numeric(
    station_data["precip"], errors="coerce", downcast="float"
)
station_data["humidity"] = pd.to_numeric(
    station_data["humidity"], errors="coerce", downcast="float"
)
station_data["wind_dir"] = pd.to_numeric(
    station_data["wind_dir"], errors="coerce", downcast="float"
)
station_data["wind_speed"] = pd.to_numeric(
    station_data["wind_speed"], errors="coerce", downcast="float"
)

# eliminamos las filas con valores faltantes
station_data = station_data.dropna()
station_data.head()

In [ ]:
station_data["next_temp"] = station_data["temp"].shift(-1)
station_data = station_data.dropna()
station_data

Hasta este punto hemos copiado el código que usamos en la primera sesión para cargar los datos y preprocesarlos.
En esta oportunidad, vamos a agregar dos datos adicionales de entrada que sera el Mes, el Dia y la Hora de la medición en forma de armónicos. Esto ayudara al modelo a capturar la estacionalidad de los datos.

In [ ]:
station_data["hour_sin"] = np.sin(2 * np.pi / (station_data.date.dt.hour + 1e-3))
station_data["hour_cos"] = np.cos(2 * np.pi / (station_data.date.dt.hour + 1e-3))
station_data["day_sin"] = np.sin(2 * np.pi / station_data.date.dt.day_of_year)
station_data["day_cos"] = np.cos(2 * np.pi / station_data.date.dt.day_of_year)
station_data["month_sin"] = np.sin(2 * np.pi / station_data.date.dt.month)
station_data["month_cos"] = np.cos(2 * np.pi / station_data.date.dt.month)

In [ ]:
station_data.head()

In [ ]:
train_size = int(len(station_data) * 0.7)
val_size = int(len(station_data) * 0.15)
test_size = len(station_data) - train_size - val_size

input_vars = [
    "temp",
    "humidity",
    "wind_speed",
    "hour_sin",
    "hour_cos",
    "day_sin",
    "day_cos",
    "month_sin",
    "month_cos",
]

train_data = station_data[input_vars].iloc[:train_size]
train_label = station_data["next_temp"].iloc[:train_size]

val_data = station_data[input_vars].iloc[train_size : train_size + val_size]
val_label = station_data["next_temp"].iloc[train_size : train_size + val_size]

test_data = station_data[input_vars].iloc[train_size + val_size :]
test_label = station_data["next_temp"].iloc[train_size + val_size :]

In [ ]:
mean = train_data.mean()
std = train_data.std()

train_data_standarized = (train_data - mean) / std
val_data_standarized = (val_data - mean) / std
test_data_standarized = (test_data - mean) / std

Ahora creamos nuestra red neuronal sin dropout como linea base.

In [ ]:
model_baseline = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(1),
    ]
)
model_baseline.build(input_shape=(None, len(input_vars)))

model_baseline.compile(optimizer="adam", loss="mse", metrics=["mae"])

history = model_baseline.fit(
    train_data_standarized,
    train_label,
    epochs=100,
    validation_data=(val_data_standarized, val_label),
    verbose=1,
)

Evaluamos el rendimiento del modelo y visualizamos las métricas de entrenamiento.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
ax[0].plot(history.history["loss"], label="train")
ax[0].plot(history.history["val_loss"], label="test")
ax[0].legend()
ax[0].set_title("loss")

ax[1].plot(history.history["mae"], label="train")
ax[1].plot(history.history["val_mae"], label="test")
ax[1].legend()
ax[1].set_title("mae")

Ahora revisamos que tan preciso es el modelo en la predicción de la temperatura.

In [ ]:
_, baseline_mae = model_baseline.evaluate(test_data_standarized, test_label, verbose=0)
print(f"Baseline MAE: {baseline_mae:.2f}")
preds_baseline = model_baseline.predict(test_data_standarized)

fig, ax = plt.subplots()
ax.plot(test_label.values, label="true")
ax.plot(preds_baseline, label="baseline")
ax.legend()

Ahora definiremos nuestra red neuronal con dropout en la capa oculta.

In [ ]:
model_dropout = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1),
    ]
)
model_dropout.build(input_shape=(None, len(input_vars)))

model_dropout.compile(optimizer="adam", loss="mse", metrics=["mae"])

history = model_dropout.fit(
    train_data_standarized,
    train_label,
    epochs=100,
    validation_data=(val_data_standarized, val_label),
    verbose=1,
)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
ax[0].plot(history.history["loss"], label="train")
ax[0].plot(history.history["val_loss"], label="test")
ax[0].legend()
ax[0].set_title("loss")

ax[1].plot(history.history["mae"], label="train")
ax[1].plot(history.history["val_mae"], label="test")
ax[1].legend()
ax[1].set_title("mae")

In [ ]:
_, dropout_mae = model_dropout.evaluate(test_data_standarized, test_label, verbose=0)
print(f"Dropout MAE: {dropout_mae:.2f}")
preds_dropout = model_dropout.predict(test_data_standarized)

fig, ax = plt.subplots()
ax.plot(test_label.values, label="true")
ax.plot(preds_dropout, label="baseline")
ax.legend()

Podemos observar como nuestro modelo con dropout logra generalizar mejor a los datos de prueba, lo que se refleja en una menor pérdida y un mayor rendimiento en la métrica de precisión.

## Early stopping

El entrenamiento de un modelo de Machine Learning puede ser un proceso costoso en términos de tiempo y recursos computacionales. Por esta razón, es importante tener en cuenta estrategias para detener el entrenamiento de un modelo cuando ya no se observa una mejora significativa en su rendimiento.

Una de las estrategias más comunes para detener el entrenamiento de un modelo es el uso de la técnica de early stopping. Esta técnica consiste en monitorear una métrica de rendimiento, como la pérdida en el conjunto de validación, y detener el entrenamiento del modelo cuando la métrica deja de mejorar.

En Keras, la técnica de early stopping se puede implementar utilizando el callback `EarlyStopping`. Este callback permite especificar la métrica a monitorear, el número de épocas para esperar antes de detener el entrenamiento y otras opciones como la paciencia y el modo de monitoreo.

```python
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')
```

Para este ejemplo vamos a usar una configuracion la misma cantidad de neuronas usadas en los ejemplos anteriores, pero con una cantidad de epocas de entrenamiento de 1000.

In [ ]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(1),
    ]
)
model.build(input_shape=(None, len(input_vars)))
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

Definimos el callback de early stopping y lo agregamos a la lista de callbacks al entrenar el modelo.

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=30, restore_best_weights=True
    )
]
history = model.fit(
    train_data_standarized,
    train_label,
    epochs=1000,
    validation_data=(val_data_standarized, val_label),
    verbose=1,
    callbacks=callbacks,
)

Podemos observar como el modelo se detiene antes de las 100 épocas, ya que no se observa una mejora significativa en la pérdida en el conjunto de validación.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
ax[0].plot(history.history["loss"], label="train")
ax[0].plot(history.history["val_loss"], label="test")
ax[0].axvline(
    x=len(history.history["loss"]) - 10,
    color="red",
    linestyle="--",
    label="early stopping",
)
ax[0].legend()
ax[0].set_title("loss")

ax[1].plot(history.history["mae"], label="train")
ax[1].plot(history.history["val_mae"], label="test")
ax[1].legend()
ax[1].set_title("mae")

In [ ]:
_, mae = model.evaluate(test_data_standarized, test_label, verbose=0)
print(f"Test MAE: {mae:.2f}")
preds = model.predict(test_data_standarized)

fig, ax = plt.subplots()
ax.plot(test_label.values, label="true")
ax.plot(preds, label="Prediction")
ax.legend()

En general, es recomendable el uso de Early Stopping en todo momento para evitar el sobreajuste de los modelos y reducir el tiempo de entrenamiento. Sin embargo, es importante tener en cuenta que el uso de Early Stopping puede llevar a detener el entrenamiento antes de que el modelo haya convergido completamente, por lo que es importante ajustar los parámetros de paciencia y modo de monitoreo para obtener los mejores resultados.

### Ejercicio

Usando la misma configuracion del modelo anterior, modifique los parametros de early stopping para que:

- Monitoree la métrica de mae en el conjunto de validación.
- Comience a monitorear después de 50 épocas.

Para ello, puede consultar la [documentación](https://keras.io/api/callbacks/early_stopping/) de Keras sobre el callback `EarlyStopping` para ver cómo configurar los parámetros adecuadamente.

In [ ]:
# code here

## Batch normalization

Batch normalization es una técnica utilizada en redes neuronales para normalizar las activaciones de una capa oculta en mini-batches de datos. Esto ayuda a estabilizar y acelerar el entrenamiento de la red, ya que evita que las activaciones se vuelvan demasiado grandes o pequeñas, lo que puede llevar a problemas de convergencia y desvanecimiento del gradiente.

En Keras, la capa de batch normalization se puede agregar a un modelo utilizando la clase `BatchNormalization`. Esta capa se puede agregar después de una capa de activación en una red neuronal y normaliza las activaciones de la capa anterior en mini-batches de datos.

```python
from tensorflow.keras.layers import BatchNormalization

model = Sequential([
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='softmax')
])
```

Nuevamente, usando la configuracion de la red neuronal anterior, vamos a agregar una capa de batch normalization después de la capa de activación en la red neuronal.

In [ ]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1),
    ]
)
model.build(input_shape=(None, len(input_vars)))
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=30, restore_best_weights=True
    )
]
history = model.fit(
    train_data_standarized,
    train_label,
    epochs=1000,
    validation_data=(val_data_standarized, val_label),
    verbose=1,
    callbacks=callbacks,
)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
ax[0].plot(history.history["loss"], label="train")
ax[0].plot(history.history["val_loss"], label="test")
ax[0].axvline(
    x=len(history.history["loss"]) - 10,
    color="red",
    linestyle="--",
    label="early stopping",
)
ax[0].legend()
ax[0].set_title("loss")

ax[1].plot(history.history["mae"], label="train")
ax[1].plot(history.history["val_mae"], label="test")
ax[1].legend()
ax[1].set_title("mae")

In [ ]:
_, mae = model.evaluate(test_data_standarized, test_label, verbose=0)
print(f"Test MAE: {mae:.2f}")
preds = model.predict(test_data_standarized)

fig, ax = plt.subplots()
ax.plot(test_label.values, label="true")
ax.plot(preds, label="Prediction")
ax.legend()

**¿Cómo afecta la capa de batch normalization al rendimiento del modelo?**

Podemos observar cómo la capa de batch normalization ayuda a estabilizar y acelerar el entrenamiento de la red, lo que se refleja en una menor pérdida y un mayor rendimiento en la métrica de precisión.

## Ejercicio

Modifique la red neuronal anterior para usar la capa de batch normalization solamente después de la capa de entrada. ¿Cómo afecta esto al rendimiento del modelo?

Ahora intente combinar la capa de batch normalization con la técnica de dropout en la red neuronal. ¿Cómo afecta esto al rendimiento del modelo? ¿El orden en que se aplican las técnicas afecta al rendimiento del modelo?

In [ ]:
# code here